# gRPC
* https://grpc.io/docs/languages/python/

In [4]:
!pip install grpcio

# Python’s gRPC tools include the protocol buffer compiler protoc and the special plugin for generating server and client code from .proto service definitions.
!pip install grpcio-tools

In [1]:
# cleanup
!pip uninstall grpcio -y
!pip uninstall grpcio-tools -y

Found existing installation: grpcio 1.68.1
Uninstalling grpcio-1.68.1:
  Successfully uninstalled grpcio-1.68.1
Found existing installation: grpcio-tools 1.68.1
Uninstalling grpcio-tools-1.68.1:
  Successfully uninstalled grpcio-tools-1.68.1


# Generate Code

In [7]:
!python3 -m grpc_tools.protoc -I./grpc --python_out=grpc_gen --pyi_out=grpc_gen --grpc_python_out=grpc_gen ./grpc/helloworld.proto

Update in `helloworld_pb2_grpc.py`:

```python
# import helloworld_pb2 as helloworld__pb2
from . import helloworld_pb2 as helloworld__pb2
```

# Server

In [ ]:
from concurrent import futures
import logging

import grpc
from grpc_gen import helloworld_pb2
from grpc_gen import helloworld_pb2_grpc


class Greeter(helloworld_pb2_grpc.GreeterServicer):
    def SayHello(self, request, context):
        return helloworld_pb2.HelloReply(message="Hello, %s!" % request.name)

t_DONE = False

def serve():
    port = "50051"
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    helloworld_pb2_grpc.add_GreeterServicer_to_server(Greeter(), server)
    server.add_insecure_port("[::]:" + port)
    server.start()
    print("Server started, listening on " + port)
    while not t_DONE:
        server.wait_for_termination(1) # 1 second
    server.stop(1)
    print("Bye.")

# if __name__ == "__main__":
#     logging.basicConfig()
#     serve()

import threading
t = threading.Thread(target=serve)
t.start()

Server started, listening on 50051
Bye.


In [3]:
t_DONE = True

# Client

In [2]:
from __future__ import print_function

import logging

import grpc
from grpc_gen import helloworld_pb2
from grpc_gen import helloworld_pb2_grpc


def run():
    # NOTE(gRPC Python Team): .close() is possible on a channel and should be
    # used in circumstances in which the with statement does not fit the needs
    # of the code.
    print("Will try to greet world ...")
    with grpc.insecure_channel("localhost:50051") as channel:
        stub = helloworld_pb2_grpc.GreeterStub(channel)
        response = stub.SayHello(helloworld_pb2.HelloRequest(name="you"))
    print("Greeter client received: " + response.message)


logging.basicConfig()
run()

Will try to greet world ...
Greeter client received: Hello, you!
